In [1]:
import pickle
import csv
import os
from pathlib import Path
from typing import Set, Tuple, NamedTuple, List, Dict, Counter, Optional

import torch
import numpy as np
from scipy.spatial import distance
from scipy.stats import spearmanr

from evaluations.euphemism import Embedding, PhrasePair  
from recomposer import Recomposer, RecomposerConfig
from decomposer import Decomposer, DecomposerConfig


torch.manual_seed(42)
np.random.seed(42)

### Load Pretrained Embedding

In [2]:
pretrained = Embedding('../../data/pretrained_word2vec/partisan_news.txt', 'plain_text')

vocab_size = 138,394, num_dimensions = 300
Loading embeddings from ../../data/pretrained_word2vec/partisan_news.txt
Done


In [7]:
stuff = Path('../../results/PN/GM2/NS10/epoch45.pt')
# stuff = Path('../../results/PN/GM2/NS5/epoch15.pt')
deno_space = Embedding(
    stuff, 'recomposer_deno', device=torch.device('cuda:0'))
cono_space = Embedding(
    stuff, 'recomposer_cono', device=torch.device('cuda:0'))

In [8]:
def tabulate(q1, q2):
    PE_cs = pretrained.cosine_similarity(q1, q2)
    DS_cs = deno_space.cosine_similarity(q1, q2)
    CS_cs = cono_space.cosine_similarity(q1, q2)
    # also get neighbor ranking?
    print(round(PE_cs, 4), round(DS_cs, 4), round(CS_cs, 4), q1, q2, sep='\t')

def tabulate_rank(q1, q2):
    print(
        pretrained.neighbor_rank(q1, q2),
        pretrained.neighbor_rank(q2, q1),
        deno_space.neighbor_rank(q1, q2),
        deno_space.neighbor_rank(q2, q1),
        cono_space.neighbor_rank(q1, q2),
        cono_space.neighbor_rank(q2, q1),
        q1, q2, sep='\t')
    
# def cf(q1, q2)
#     pretrained.nearest_neighbor(q1)
#     model.nearest_neighbor(q1)
#     print('\n')
#     pretrained.nearest_neighbor(q2)
#     model.nearest_neighbor(q2)

In [9]:
cherry_pairs = [
    # Luntz Report, all GOP euphemisms
    ('government', 'washington'),
    # ('private_account', 'personal_account'),
    # ('tax_reform', 'tax_simplification'),
    ('estate_tax', 'death_tax'),
    ('capitalism', 'free_market'),  # global economy, globalization
    # ('outsourcing', 'innovation'),  # "root cause" of outsourcing, regulation
    ('undocumented_workers', 'illegal_aliens'),  # OOV undocumented_workers
    ('foreign_trade', 'international_trade'),  # foreign, global all bad
    # ('drilling_for_oil', 'exploring_for_energy'),
    # ('drilling', 'energy_exploration'),
    # ('tort_reform', 'lawsuit_abuse_reform'),
    # ('trial_lawyer', 'personal_injury_lawyer'),  # aka ambulance chasers
    # ('corporate_transparency', 'corporate_accountability'),
    # ('school_choice', 'parental_choice'),  # equal_opportunity_in_education
    # ('healthcare_choice', 'right_to_choose')

    # Own Cherries
    ('public_option', 'government_run'),
    ('political_speech', 'campaign_spending'),  # hard example
    ('cut_taxes', 'supply_side'),  # OOV supplyside
    
    # Own Cherries
    ('public_option', 'government_run'),
    ('political_speech', 'campaign_spending'),  # hard example
    ('cut_taxes', 'supply_side'),  
    ('voodoo', 'supply_side'),
    
    # large vocabulary
#     ('star_wars', 'strategic_defense_initiative'),
    ('socialized_medicine', 'single_payer'),
#     ('cap_and_trade', 'national_energy_tax'),
    ('waterboarding', 'interrogation'),
    ('tax_expenditures', 'spending_programs'),
#     ('nuclear_option', 'constitutional_option'),
]



In [10]:
for q1, q2 in cherry_pairs:
    tabulate_rank(q1, q2)

18641	37128	44	1209	4304	7229	government	washington
1	8	229	231	3807	2063	estate_tax	death_tax
109	13	19284	11132	20	6	capitalism	free_market
71	14	121388	755	42778	11727	undocumented_workers	illegal_aliens
11095	15	91253	2677	119	3	foreign_trade	international_trade
39	30	22	10	243	7	public_option	government_run
1523	410	93	65457	30122	65192	political_speech	campaign_spending
17692	14059	32468	64603	21505	32959	cut_taxes	supply_side
39	30	22	10	243	7	public_option	government_run
1523	410	93	65457	30122	65192	political_speech	campaign_spending
17692	14059	32468	64603	21505	32959	cut_taxes	supply_side
38137	6000	4612	30617	77372	94567	voodoo	supply_side
1192	6	105002	363	610	181	socialized_medicine	single_payer
11	15	123	159	17	7	waterboarding	interrogation
23	557	95	8076	32448	45994	tax_expenditures	spending_programs


In [ ]:
for q1, q2 in cherry_pairs:
    tabulate(q1, q2)

In [ ]:
cf('undocumented_workers', 'illegal_aliens')

In [ ]:
cf('estate_tax', 'death_tax')

In [ ]:
# Denotation Space: Want to see stuff from other party
# model.embedding = model.deno_embed
# q1 = 'universal_health_care'  # OOV
# q2 = 'socialized_medicine'
model = deno_space
q1 = 'singlepayer'
q2 = 'governmentrun'
print(pretrained.cosine_similarity(q1, q2))
print(model.cosine_similarity(q1, q2))
pretrained.nearest_neighbor(q1)
model.nearest_neighbor(q1)
print('\n')
pretrained.nearest_neighbor(q2)
model.nearest_neighbor(q2)

In [ ]:
model = deno_space
q1 = 'public_option'
q2 = 'governmentrun'
print(pretrained.cosine_similarity(q1, q2))
print(model.cosine_similarity(q1, q2))
pretrained.nearest_neighbor(q1)
model.nearest_neighbor(q1)
print('\n')
pretrained.nearest_neighbor(q2)
model.nearest_neighbor(q2)

In [ ]:
model = deno_space
q1 = 'independent_expenditures'
q2 = 'political_speech'
print(pretrained.cosine_similarity(q1, q2))
print(model.cosine_similarity(q1, q2))
pretrained.nearest_neighbor(q1)
model.nearest_neighbor(q1)
print('\n')
pretrained.nearest_neighbor(q2)
model.nearest_neighbor(q2)

In [ ]:
model = deno_space
q1 = 'tax_breaks'
q2 = 'tax_relief'
print(pretrained.cosine_similarity(q1, q2))
print(model.cosine_similarity(q1, q2))
pretrained.nearest_neighbor(q1)
model.nearest_neighbor(q1)
print('\n')
pretrained.nearest_neighbor(q2)
model.nearest_neighbor(q2)

In [ ]:
model = deno_space
q1 = 'socialized_medicine' # 'obamacare'
q2 = 'health_care_reform'
print(pretrained.cosine_similarity(q1, q2))
print(model.cosine_similarity(q1, q2))
pretrained.nearest_neighbor(q1)
model.nearest_neighbor(q1)
print('\n')
pretrained.nearest_neighbor(q2)
model.nearest_neighbor(q2)

### Connotation Space: Want to see unrelated random entities of the same party

In [ ]:
model = cono_space
q1 = 'gun_control'
q2 = 'illegal_aliens'
print(pretrained.cosine_similarity(q1, q2))
print(model.cosine_similarity(q1, q2))
pretrained.nearest_neighbor(q1)
model.nearest_neighbor(q1)
print('\n')
pretrained.nearest_neighbor(q2)
model.nearest_neighbor(q2)

In [ ]:
model = cono_space
q1 = 'wall_street_reform'
q2 = 'civil_rights'
print(pretrained.cosine_similarity(q1, q2))
print(model.cosine_similarity(q1, q2))
pretrained.nearest_neighbor(q1)
model.nearest_neighbor(q1)
print('\n')
pretrained.nearest_neighbor(q2)
model.nearest_neighbor(q2)

In [ ]:
model = cono_space
q1 = 'nuclear_arms_race'
q2 = 'credit_card'
print(pretrained.cosine_similarity(q1, q2))
print(model.cosine_similarity(q1, q2))
pretrained.nearest_neighbor(q1)
model.nearest_neighbor(q1)
print('\n')
pretrained.nearest_neighbor(q2)
model.nearest_neighbor(q2)

In [ ]:
model = cono_space
q = 'national_energy_tax'
pretrained.nearest_neighbor(q)
model.nearest_neighbor(q)

In [ ]:
pretrained.cosine_similarity('tariff', 'employers')